In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import LassoSelector
from matplotlib.path import Path
from ipywidgets import FileUpload, Button, Output, FloatText
from IPython.display import display
import io

# Enable interactive Matplotlib mode
%matplotlib widget

# Create widgets
upload = FileUpload(accept='.jpg,.png', multiple=False)
button_process = Button(description="Process Image")
scale_input = FloatText(description="Scale (pixels per inch):", value=100.0)
button_measure = Button(description="Measure Area")
output = Output()

# Global variables
image = None
image_rgb = None
fig, ax = None, None
lasso = None
selected_pixels = []
scale_pixels_per_inch = None
mask = None  # Store the mask for the selection

def process_image(upload_widget):
    """Reads an uploaded image file and converts it to a NumPy array."""
    if not upload_widget.value:
        print("Error: No image uploaded.")
        return None

    print("Debug: upload.value type:", type(upload_widget.value))
    print("Debug: upload.value content:", upload_widget.value)

    file_data = next(iter(upload_widget.value))  # Extract file data
    file_bytes = file_data['content']

    # Convert byte content into a NumPy array
    image_array = np.frombuffer(file_bytes, np.uint8)

    # Decode image using OpenCV
    img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    if img is None:
        print("Error: Failed to decode image.")
        return None

    return img

def on_button_process(b):
    """Handles button click to process the uploaded image."""
    global image, image_rgb, fig, ax, lasso, selected_pixels, scale_pixels_per_inch, mask

    with output:
        output.clear_output()

        # Process the uploaded image
        image = process_image(upload)

        if image is None:
            print("Please upload a valid image file.")
            return

        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        selected_pixels = []
        scale_pixels_per_inch = None
        mask = None

        # Create a new persistent figure for selection
        fig, ax = plt.subplots()
        ax.imshow(image_rgb)
        ax.set_title("Enter Scale (pixels per inch) and Click 'Measure Area'")
        ax.axis('off')

        plt.show()

def on_button_measure(b):
    """Activates Lasso Tool for selecting a region."""
    global fig, ax, lasso, selected_pixels

    if image_rgb is None:
        print("Error: No image to process. Upload and process an image first.")
        return

    selected_pixels = []

    # Use existing figure to ensure Lasso remains active
    ax.set_title("Use Lasso Tool to Select Area and Release Mouse")
    global lasso
    lasso = LassoSelector(ax, on_lasso_select)

    plt.draw()

def on_lasso_select(verts):
    """Handles Lasso Tool selection and computes area."""
    global image_rgb, scale_pixels_per_inch, selected_pixels, mask

    # Store the selected region as a path
    path = Path(verts)

    # Create a new mask for this selection
    mask = np.zeros(image_rgb.shape[:2], dtype=np.uint8)
    for y in range(mask.shape[0]):
        for x in range(mask.shape[1]):
            if path.contains_point((x, y)):
                mask[y, x] = 255  # Mark the selected region

    # Calculate the area in pixels
    pixel_area = np.sum(mask) / 255  # Count white pixels

    # Convert to inches² using scale
    scale_pixels_per_inch = scale_input.value  # Get user-defined scale
    if scale_pixels_per_inch > 0:
        inch_area = pixel_area / (scale_pixels_per_inch ** 2)
        print(f"✅ Measured Area: {inch_area:.2f} square inches")

        # Display the selected region immediately with its calculated area
        fig, ax = plt.subplots()
        ax.imshow(cv2.bitwise_and(image_rgb, image_rgb, mask=mask))
        ax.set_title(f"Selected Area: {inch_area:.2f} in²")
        ax.axis('off')
        plt.show()
    else:
        print("❌ Error: Scale must be greater than zero.")

# Display widgets
button_process.on_click(on_button_process)
button_measure.on_click(on_button_measure)

display(upload, button_process, scale_input, button_measure, output)
